In [1]:
import requests
from bs4 import BeautifulSoup
import os
import time
try:
    from urllib.request import urlretrieve
except ImportError:
    from urllib import urlretrieve
import xml.sax
from sklearn import svm
import subprocess
import mwparserfromhell
import json
from collections import Counter
from itertools import chain
import numpy as np
import random
from keras.models import Model
from keras.layers import Embedding, Input, Reshape
from keras.layers.merge import Dot
from sklearn.linear_model import LinearRegression

Using TensorFlow backend.


In [ ]:
index = requests.get('https://dumps.wikimedia.org/enwiki/').text

In [ ]:
soup_index = BeautifulSoup(index, 'html.parser')

In [ ]:
dumps = [a['href'] for a in soup_index.find_all('a') 
             if a.has_attr('href') and a.text[:-1].isdigit()]
dumps

In [ ]:
for dump_url in sorted(dumps, reverse=True):
  print(dump_url)
  dump_html = index = requests.get('https://dumps.wikimedia.org/enwiki/' + dump_url).text
  soup_dump = BeautifulSoup(dump_html, 'html.parser')
  pages_xml = [a['href'] for a in soup_dump.find_all('a') 
             if a.has_attr('href') and a['href'].endswith('-pages-articles.xml.bz2')]
  if pages_xml:
    break
  time.sleep(0.8)

In [ ]:
wikipedia_dump = pages_xml[0].rsplit('/')[-1]
if not os.path.isfile(wikipedia_dump):
    url = 'https://dumps.wikimedia.org/' + pages_xml[0]
    urlretrieve(url, wikipedia_dump)


In [ ]:
def process_article(title, text):
  rotten = [(re.findall('\d\d?\d?%', p), re.findall('\d\.\d\/\d+|$', p), p.lower().find('rotten tomatoes')) for p in text.split('\n\n')]
  rating = next(((perc[0], rating[0]) for perc, rating, idx in rotten if len(perc) == 1 and idx > -1), (None, None))
  wikicode = mwparserfromhell.parse(text)
  film = next((template for template in wikicode.filter_templates() 
               if template.name.strip().lower() == 'infobox film'), None)
  if film:
    properties = {param.name.strip_code().strip(): param.value.strip_code().strip() 
                  for param in film.params
                  if param.value.strip_code().strip()
                 }
    links = [x.title.strip_code().strip() for x in wikicode.filter_wikilinks()]
    return (title, properties, links) + rating

In [ ]:
class WikiXmlHandler(xml.sax.handler.ContentHandler):
  def __init__(self):
    xml.sax.handler.ContentHandler.__init__(self)
    self._buffer = None
    self._values = {}
    self._movies = []
    self._curent_tag = None

  def characters(self, content):
    if self._curent_tag:
      self._buffer.append(content)

  def startElement(self, name, attrs):
    if name in ('title', 'text'):
      self._curent_tag = name
      self._buffer = []

  def endElement(self, name):
    if name == self._curent_tag:
      self._values[name] = ' '.join(self._buffer)

    if name == 'page':
      movie = process_article(**self._values)
      if movie:
        self._movies.append(movie)


In [ ]:
parser = xml.sax.make_parser()
handler = WikiXmlHandler()
parser.setContentHandler(handler)
for line in subprocess.Popen(['bzcat'], stdin=open(wikipedia_dump), stdout=subprocess.PIPE).stdout:
  try:
    parser.feed(line)
  except StopIteration:
    break

In [ ]:
with open('wp_movies.ndjson', 'wt') as fout:
  for movie in handler._movies:
    fout.write(json.dumps(movie) + '\n')